In [1]:
import openai
import os
from openai import OpenAI
import json
# gpt 请求接口
openai_api_key=""
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
)
# 基于 prompt 生成文本
def get_completion(prompt_sys,prompt_user, model="gpt-4-1106-preview"):
    messages = [
        {"role": "system","content": prompt_sys},
        {"role": "user",  "content": prompt_user}
        ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        # temperature=0.9,  # 模型输出的随机性，0 表示随机性最小
    )
    # client.chat.completions.create
    return response.choices[0].message.content.strip()

In [2]:
prompt_sys="""
你是一名营销文案专家，接下来会获取到``写作要求``与对应生成的``营销文案``
请严格的站在实际应用的角度以及写作要求对营销文案进行评分。5分制,并说明打分原因。
评分维度如下：
```
平台调性（写作要求的平台调性和营销文案所展现的平台调性是否对应）、
品牌信息（写作要求的品牌信息和营销文案的品牌信息是否对应）、
品牌特点/卖点（写作要求的品牌特点/卖点和营销文案的品牌特点/卖点是否对应）、
受众群体（写作要求的受众群体和营销文案的受众群体是否对应）、
可控字数（如果写作要求未提及字数限制，则按照标准社交媒体文案进行评分）、
设定发文者视角（写作要求的设定发文者视角和营销文案的发文者视角是否相同）、
文案标题、
故事性描述（营销文案是否提及一个生动有趣的故事）、
表情/网络用语、
文案通用能力、
文案风格（写作要求的文案风格和营销文案的文案风格是否相同）、
文本生成（营销文案的完整性，连贯性和逻辑性）、
文本创作（营销文案的创新度）、
安全能力
使用场景（营销文案在特定的场景下与写作要求所指定的目标受众的需求和情境相符合）
```
对于任意评分维度，评分标准如下：
```
* 0分：评分维度错误，不明显，无法识别。
* 1分：评分维度比较模糊，但可以勉强识别。
* 2分：评分维度比较清晰，但还有待完善。
* 3分：评分维度比较清晰，并且能够比较准确地传达品牌形象。
* 4分：评分维度非常清晰，并且能够非常准确地传达品牌形象。
* 5分：评分维度非常清晰，并且能够非常准确地传达品牌形象，并且能够让用户产生共鸣。

```
输出要求:返回各个维度的分数与打分理由
输出示例:
    {
        "平台调性":["分数","理由"],
        "品牌信息":["分数","理由"],
        ...
    }

"""


In [3]:
import pandas as pd
df = pd.read_excel("小红书验收.xlsx")
df

,prompt,answer,平台调性,品牌信息,品牌特点/卖点,受众群体,可控字数,设定发文者视角,文案标题,故事性描述,表情/网络用语,文案通用能力,文案风格,文本生成,文本创作,安全能力,平均分,评价
0,你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容，站在婚庆摄影的视角上推...,标题:💥小米 Civi 3: 信号强，通话清晰，随时随地的移动信号塔！📞\n内容:💥想要在婚...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,帮我生成一下小米MIX FOLD 2营销文案，小红书文风，站在工作者的视角上推荐产品，以第一...,标题: 小米 MIX FOLD 2: 出差神器，让你的旅途无忧🚀\n内容: 大家好，我是一名...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容，站在实用主义的视角上推...,标题: 小米12T Pro: 商务人士的最佳拍档，随时随地，畅享智能生活📱\n内容: 大家好...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,请站在营销专员的视角上写一篇小红书文案，推荐一下苹果15Pro，加入故事性描写，突出品牌卖点...,标题:🍎苹果15Pro: 摄影达人的首选，让你的照片更清晰、更生动！📸\n内容: 嗨，苹果手...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,结合以下信息帮我写一段小红书文案，产品是华为P60，站在产品可靠的视角上推荐产品，加入故事性...,"标题:🌳🦌探索世界，用华为 P60记录美好时光！🌟🦌\n内容:🌟🦌最近我收到了华为 P60,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,请写一篇推荐一下华为Mate X5手机的小红书营销文案\n,"标题:🌟华为 Mate X5: 折叠新境界，生活好伙伴🌟\n内容:Hey,亲爱的小伙伴们！👋...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,请站在健康的视角上帮我推荐一下苹果15这个产品，输出小红书文案，它具有对于身体健康的预警，监...,"标题: 苹果15,健康生活的贴心伴侣！\n内容:Hello,朋友们！👋今天我要给大家分享一款...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,你是一位小红书爆款写作专家，请你根据以下内容创作小红书文风的爆款内容，加入故事性描写来引发读...,标题: 一加手机: 游戏玩家的持久伙伴！🎮🔋\n内容:🔥🔥🔥小红书的朋友们，今天我要给大家推...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,写一段小红书文案，推荐一下苹果14 ProMax，加入第一人称故事性描写，手机卖点：大容量存...,标题:🍎苹果14 ProMax: 大容量与卓越性能的完美融合🍎\n内容:💪果粉们，让你们眼前...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,你是一位小红书爆款写作专家，在文案中加入相关标签或话题来突出重点内容。\n主题：推荐一下苹果...,"标题: 苹果13 Pro Max: 深海潜水员的安全守护者！\n内容:Hello,亲爱的深海...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
result = """
{
    "平台调性":["5","文案以小红书的平台调性撰写，适当使用了表情符号来吸引注意力，并在内容中加入了相关标签和话题来引导读者进行互动。"],
    "品牌信息":["4","文案清晰地提及了小米 Civi 3，准确传达了品牌信息。"],
    "品牌特点/卖点":["5","文案很好地展现了品牌的卖点：信号好、通话质量高、适合嘈杂环境、全新设计风格以及超轻薄便携，符合要求。"],
    "受众群体":["3","针对平价购买者的需求，文案强调了产品在多个情景中的稳定信号和通话质量，但未明确强调价格优势。"],
    "可控字数":["5","文案的长度在300字左右，符合要求。"],
    "设定发文者视角":["4","从婚庆摄影的角度出发，推介产品使用的情景与要求一致，但文案中缺少摄影具体的视角相关内容。"],
    "文案标题":["5","标题使用了表情符号和直接的语言，很好地吸引了受众的注意力。"],
    "故事性描述":["3","文案中描述了产品在不同场景下的应用情境，有一定的故事性，但故事情节并不丰富。"],
    "表情/网络用语":["5","适当且合理地运用了表情和网络热词，符合小红书平台的风格。"],
    "文案通用能力":["4","文案结构清晰，信息点突出，从多个角度介绍产品特点。"],
    "文案风格":["4","文案整体风格简洁明了，但使用了较多表情符号，略微减弱了简洁性。"],
    "文本生成":["4","文案内容完整，逻辑流畅，但在展示产品特点时的描述略显重复。"],
    "文本创作":["3","虽然文案贴合营销目的，但在创新性方面并无过多突出之处。"],
    "安全能力":["5","文案内容没有涉及敏感或不当言论，符合发布安全规则。"],
    "使用场景":["5","文案很好地将产品的使用场景与目标受众的需求结合起来，婚礼现场、工作室、车上等场景均有提及。"]
}
"""

In [5]:
for i in range(len(df)):
    user_prompt = "写作要求: \n" + df.iloc[i,0] + "\n 营销文案: \n" + df.iloc[i,1]
    success = 0
    while(success == 0):
        try:
            result = get_completion(prompt_sys=prompt_sys,prompt_user=user_prompt)
            print(i)
            print(result)
            result = json.loads(result)
            a = 2
            comment = ""
            for item in result:
                df.iloc[i,a] = int(result[item][0])
                a += 1
                comment += str(item) + ":" + str(result[item][1]) + "。\n"
            df.iloc[i,16] = df.iloc[i, 2:16].mean() 
            df.iloc[i,17] = comment
            success = 1
        except:
            print("响应时间过长")

    

0
{
    "平台调性":["5","文案运用了小红书上常见的表情符号，加入了故事性描述，并通过引人入胜的开场和紧扣主题的内容展现了非常符合小红书平台调性的文风。"],
    "品牌信息":["4","文案成功地提到了品牌名称——小米Civi 3，并通过全文强调其信号和通话质量，较好地传达了品牌信息。"],
    "品牌特点/卖点":["4","明确提出了小米Civi 3的卖点：信号好、通话质量高、全新设计风格、超轻薄，且将这些优势贯穿全文。"],
    "受众群体":["3","适当考虑到了平价购买者对信号和通话质量的需求，尽管文案很少涉及价格话题，但关注产品核心功能的表述符合预期的受众群体。"],
    "可控字数":["5","文案在300字左右，满足了字数要求。"],
    "设定发文者视角":["4","文案成功地站在婚庆摄影的视角，提出在婚礼现场、工作室、车上无论何处都能使用小米Civi 3，与设定发文者视角相匹配。"],
    "文案标题":["5","标题使用了表情符号和夸张的用语，非常适合吸引小红书用户的注意，且直击产品特点。"],
    "故事性描述":["3","文案尝试用故事性描述来吸引用户，让人想象在不同场景下使用产品的情况，但故事性可以进一步加强，提高用户的感情共鸣。"],
    "表情/网络用语":["5","文案很好地使用了表情和网络用语，符合小红书用户习惯和平台风格。"],
    "文案通用能力":["4","文案结构清晰，信息层层递进，通用能力强，但在故事性和情感共鸣上还有提升空间。"],
    "文案风格":["4","整体上展现了简洁明了风格的写作，直接点出产品特点，语言简洁有力，但仍可以进一步精简以提高简洁度。"],
    "文本生成":["4","文案内容完整，逻辑清楚且连贯，能够清晰明了地传递出所要推广的产品信息。"],
    "文本创作":["3","文案较好地运用了小红书平台的写作风格和传播方式，但在内容创新度上较为一般，未能提供独特视角或创意表达。"],
    "安全能力":["5","文案内容健康，不存在违规或敏感信息，具有良好的安全性。"],
    "使用场景":["4","将产品放置在婚庆现场、工作室和车上等多个场景中，清晰展现了使用情境，与营销目的和指定的场景相匹配，但可以加

In [10]:
df.to_excel("/home/kit/kit/text_eval/1_30_output.xlsx")